In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os


# Import API key
from api_keys import g_key

In [2]:
# Trying to get map to show
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!jupyter nbextension enable --py --sys-prefix gmaps

Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
Config option `kernel_spec_manager_class` not recognized by `EnableNBExtensionApp`.
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [3]:
# Reading in CSV file created from WeatherPy data and turning into a data frame
file = "WeatherData.csv"

file_df = pd.read_csv(file)
file_df = file_df.drop(columns=["Unnamed: 0"])
file_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bilibino,68.05,166.44,-30.48,99,81,2.93,RU,1609813374
1,Cherskiy,68.75,161.30,-31.34,78,84,5.26,RU,1609813375
2,San Cristobal,7.77,-72.22,63.19,45,92,2.37,VE,1609813376
3,Tegul'det,57.31,88.17,-24.34,0,86,3.27,RU,1609813378
4,Dingli,35.86,14.38,52.00,20,61,6.93,MT,1609813379
...,...,...,...,...,...,...,...,...,...
556,Tuy Hoa,13.08,109.30,75.49,76,90,13.82,VN,1609813994
557,Tamworth,52.63,-1.70,35.60,90,80,10.29,GB,1609813815
558,Lixouri,38.20,20.43,54.10,23,66,9.15,GR,1609813996
559,Alcañiz,41.05,-0.13,30.99,10,73,11.99,ES,1609813997


In [22]:
# Configuring gmaps
gmaps.configure(api_key=g_key)

In [24]:
# Setting variable to the values of the cordinates
locations = file_df[["Latitude", "Longitude"]].astype(float)
locations

# Dropping null values
file_df = file_df.dropna()

In [25]:
# Setting variable to the humidity values
humidity = file_df["Humidity"].astype(float)

# Creating layer and displaying map
m = gmaps.figure()
layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=max(file_df["Humidity"]),
                                   point_radius=2)
m.add_layer(layer)
m

Figure(layout=FigureLayout(height='420px'))

In [52]:
# Using loc and conditionals to filter and find the ideal cities
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Cloudiness below 30%
ideal_cities_df = file_df.loc[((file_df["Max Temp"] < 80) & (file_df["Max Temp"] > 70)) & (file_df["Wind Speed"] < 10) & (file_df["Cloudiness"] < 30)]

ideal_cities_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
171,Dinguiraye,11.30,-10.72,72.72,0,28,3.27,GN,1609813561
426,Arlit,18.74,7.39,72.00,0,22,5.03,NE,1609813849
538,Solenzo,12.18,-4.08,73.53,0,24,5.88,BF,1609813975


In [15]:
# Creating hotel data frame that looks at just the city, latitude, longitude, and country
hotel_df = file_df[["City", "Latitude", "Longitude", "Country"]]
hotel_df

# Adding column for the hotels and setting to a empty string
hotel_df["Hotel"] = ""
hotel_df

/Users/matthewpadgett/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,City,Latitude,Longitude,Country,Hotel
0,Bilibino,68.05,166.44,RU,
1,Cherskiy,68.75,161.30,RU,
2,San Cristobal,7.77,-72.22,VE,
3,Tegul'det,57.31,88.17,RU,
4,Dingli,35.86,14.38,MT,
...,...,...,...,...,...
556,Tuy Hoa,13.08,109.30,VN,
557,Tamworth,52.63,-1.70,GB,
558,Lixouri,38.20,20.43,GR,
559,Alcañiz,41.05,-0.13,ES,


In [31]:
# Dictionary to hold the parameters needed when searching
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# For loop to iterate through the row of the hotel data frame
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # Adding to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # URL used to search with
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make API request to JSON
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    
    # Setting variable to hold the results
    results = response["results"]
    
    # Try statement to use the for loop goes through each city
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel"] = results[0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: Bilibino.
Closest hotel is Gostinitsa Severyanka.
------------
Retrieving Results for Index 1: Cherskiy.
Closest hotel is Gostinitsa.
------------
Retrieving Results for Index 2: San Cristobal.
Closest hotel is Pirineos.
------------
Retrieving Results for Index 3: Tegul'det.
Closest hotel is Gostinitsa Kedr.
------------
Retrieving Results for Index 4: Dingli.
Closest hotel is The Xara Palace Relais & Chateaux.
------------
Retrieving Results for Index 5: Ushuaia.
Closest hotel is Albatros Hotel.
------------
Retrieving Results for Index 6: Busselton.
Closest hotel is Observatory Guest House.
------------
Retrieving Results for Index 7: Castro.
Closest hotel is CHACARA BAILLY.
------------
Retrieving Results for Index 8: Vila Velha.
Closest hotel is Hotel Vitória Palace.
------------
Retrieving Results for Index 9: East London.
Closest hotel is Tu Casa.
------------
Retrieving Results for Index 10: Severo-Kuril'sk.
Closest hotel is Kak Doma.
-----------

Closest hotel is Scoop Guest House.
------------
Retrieving Results for Index 88: Kudat.
Closest hotel is Dream Garden Hotel.
------------
Retrieving Results for Index 89: Portland.
Closest hotel is The Bidwell Marriott Portland.
------------
Retrieving Results for Index 90: Ulladulla.
Closest hotel is Sandpiper Motel.
------------
Retrieving Results for Index 91: Tuktoyaktuk.
Closest hotel is End of The Road Ltd..
------------
Retrieving Results for Index 92: Hambantota.
Closest hotel is Bungalow 63.
------------
Retrieving Results for Index 93: Urucará.
Closest hotel is Pousada Maria Bonita.
------------
Retrieving Results for Index 94: Torbay.
Closest hotel is Gallows Cove.
------------
Retrieving Results for Index 95: Roebourne.
Closest hotel is Harding River Caravan Park.
------------
Retrieving Results for Index 96: Dali.
Closest hotel is Dali Travelling With Hostel.
------------
Retrieving Results for Index 97: Rāmpur.
Closest hotel is OYO 38687 Hotel Shivansh.
------------
Retr

Closest hotel is Hotel Star inn Peniche.
------------
Retrieving Results for Index 175: Biak.
Closest hotel is Penginapan Kim.
------------
Retrieving Results for Index 176: Tutzing.
Closest hotel is Marina Hotel am Starnberger See.
------------
Retrieving Results for Index 177: Kaitangata.
Closest hotel is Kaitangata Motor Camp.
------------
Retrieving Results for Index 178: Souillac.
Closest hotel is Bils' Villa.
------------
Retrieving Results for Index 179: Sambava.
Closest hotel is Melrose.
------------
Retrieving Results for Index 180: Padre Bernardo.
Missing field/result... skipping.
------------
Retrieving Results for Index 181: Clyde River.
Closest hotel is Qamaq Hotel.
------------
Retrieving Results for Index 182: Altay.
Closest hotel is Jinqiao Hotel.
------------
Retrieving Results for Index 183: Saint Paul Harbor.
Closest hotel is Best Western Kodiak Inn and Convention Center.
------------
Retrieving Results for Index 184: Marawi.
Closest hotel is Derogongan Residence.
--

------------
Retrieving Results for Index 261: Husavik.
Closest hotel is Bjarnabúð.
------------
Retrieving Results for Index 262: Longyearbyen.
Closest hotel is Radisson Blu Polar Hotel, Spitsbergen.
------------
Retrieving Results for Index 263: Chyorny Yar.
Closest hotel is Stanitsa Chernoyarskaya.
------------
Retrieving Results for Index 264: Shizunai-furukawachō.
Closest hotel is Shizunai Eclipse Hotel.
------------
Retrieving Results for Index 265: Batagay-Alyta.
Missing field/result... skipping.
------------
Retrieving Results for Index 266: Hong Kong.
Closest hotel is Mini Hotel Central.
------------
Retrieving Results for Index 267: Sundsvall.
Closest hotel is Clarion Collection Hotel Grand Sundsvall.
------------
Retrieving Results for Index 268: Ketchikan.
Closest hotel is Snows Cove Lodge.
------------
Retrieving Results for Index 269: Saint-Augustin.
Missing field/result... skipping.
------------
Retrieving Results for Index 270: Athens.
Closest hotel is Novotel Athenes.


Closest hotel is Wifstavarfs Herrgård.
------------
Retrieving Results for Index 349: Meulaboh.
Closest hotel is Kost Pria.
------------
Retrieving Results for Index 350: Yumen.
Missing field/result... skipping.
------------
Retrieving Results for Index 351: Abu Dhabi.
Closest hotel is Beach Rotana.
------------
Retrieving Results for Index 352: Eureka.
Closest hotel is Eureka Inn Trademark By Wyndham.
------------
Retrieving Results for Index 353: Byron Bay.
Closest hotel is Backpackers Inn on the Beach.
------------
Retrieving Results for Index 354: Carroll.
Closest hotel is Best Western Westminster Hotel.
------------
Retrieving Results for Index 355: Novyy Urgal.
Missing field/result... skipping.
------------
Retrieving Results for Index 356: Alyangula.
Closest hotel is Groote Eylandt Lodge.
------------
Retrieving Results for Index 357: Santa Cruz de la Sierra.
Closest hotel is Hotel Cortez.
------------
Retrieving Results for Index 358: Manta.
Closest hotel is Balandra Hotel.
---

Closest hotel is Ambassador.
------------
Retrieving Results for Index 436: Heyang.
Missing field/result... skipping.
------------
Retrieving Results for Index 437: Mahon.
Closest hotel is ARTIEM Capri.
------------
Retrieving Results for Index 438: Aklavik.
Closest hotel is Aklavik Inn.
------------
Retrieving Results for Index 439: Pangody.
Closest hotel is Elizaveta Shevchuk's House.
------------
Retrieving Results for Index 440: Qasigiannguit.
Closest hotel is Hotel Discobay.
------------
Retrieving Results for Index 441: Torquay.
Closest hotel is The Grand Hotel.
------------
Retrieving Results for Index 442: Pāsighāt.
Closest hotel is Siang Guest House.
------------
Retrieving Results for Index 443: Omsukchan.
Closest hotel is Zori.
------------
Retrieving Results for Index 444: Tsimmerbude.
Closest hotel is Gostinitsa-Kafe, Alena.
------------
Retrieving Results for Index 445: Pulaski.
Closest hotel is DoubleTree by Hilton Hotel Little Rock.
------------
Retrieving Results for I

Closest hotel is Volcano Sea View Tree House.
------------
Retrieving Results for Index 522: Noshiro.
Closest hotel is Azuma Ryokan.
------------
Retrieving Results for Index 523: Benghazi.
Closest hotel is Tibesti Hotel.
------------
Retrieving Results for Index 524: Hanko.
Closest hotel is Hotel Villa Maija.
------------
Retrieving Results for Index 525: Mitú.
Missing field/result... skipping.
------------
Retrieving Results for Index 526: Gijang.
Closest hotel is Brown Dot Gijang Yeonhwari.
------------
Retrieving Results for Index 527: Totness.
Closest hotel is STAATS LOGEERGEBOUW TOTNESS.
------------
Retrieving Results for Index 528: Santo Antônio do Içá.
Closest hotel is Hotel Fogaça.
------------
Retrieving Results for Index 529: Lake Shasta.
Closest hotel is Shasta Dam Motel.
------------
Retrieving Results for Index 530: Richards Bay.
Closest hotel is BON Hotel Waterfront Richards Bay.
------------
Retrieving Results for Index 531: Hervey Bay.
Closest hotel is Shelly Bay Reso

In [38]:
hotel_df

,City,Latitude,Longitude,Country,Hotel
0,Bilibino,68.05,166.44,RU,Gostinitsa Severyanka
1,Cherskiy,68.75,161.30,RU,Gostinitsa
2,San Cristobal,7.77,-72.22,VE,Pirineos
3,Tegul'det,57.31,88.17,RU,Gostinitsa Kedr
4,Dingli,35.86,14.38,MT,The Xara Palace Relais & Chateaux
...,...,...,...,...,...
556,Tuy Hoa,13.08,109.30,VN,CenDeluxe Hotel Tuy Hoa
557,Tamworth,52.63,-1.70,GB,Drayton Manor Hotel
558,Lixouri,38.20,20.43,GR,Summery Hotel
559,Alcañiz,41.05,-0.13,ES,Hotel Ciudad de Alcañiz


In [61]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)

In [72]:
# Adding layer to map and display figure
m.add_layer(markers)
m.add_layer(hotel_layer)
m.add_layer(layer)
m

Figure(layout=FigureLayout(height='420px'))

In [63]:
# Creating new data frame to just look at the ideal city data
hotel2_df = ideal_cities_df[["City", "Latitude", "Longitude", "Country"]]
hotel2_df

# Adding column for the hotels and setting to a empty string
hotel2_df["Hotel"] = ""
hotel2_df

/Users/matthewpadgett/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,City,Latitude,Longitude,Country,Hotel
171,Dinguiraye,11.30,-10.72,GN,
426,Arlit,18.74,7.39,NE,
538,Solenzo,12.18,-4.08,BF,


In [64]:
# Dictionary to hold the parameters needed when searching
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# For loop to iterate through the row of the second hotel data frame
for index, row in hotel2_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # Adding to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # URL used to search with
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make API request to JSON
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    
    # Setting variable to hold the results
    results = response["results"]
    
    # Try statement to use the for loop goes through each city
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel2_df.loc[index, "Hotel"] = results[0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 171: Dinguiraye.
Missing field/result... skipping.
------------
Retrieving Results for Index 426: Arlit.
Closest hotel is Hôtel Telwa Bungalow.
------------
Retrieving Results for Index 538: Solenzo.
Missing field/result... skipping.
------------


/Users/matthewpadgett/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [67]:
# Using the template add the hotel marks to the heatmap
info_box_template2 = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
ideal_hotel_info = [info_box_template2.format(**row) for index, row in hotel2_df.iterrows()]
locations = hotel2_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)

# Create hotel symbol layer
hotel_layer2 = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=ideal_hotel_info
)

In [70]:
# Adding layers that are specifc to the hotel found in the ideal cities
# Also add the humdity layer
m2 = gmaps.figure()
m2.add_layer(markers)
m2.add_layer(hotel_layer2)
m2.add_layer(layer)
m2

Figure(layout=FigureLayout(height='420px'))